# Повыходный отчет с расчетом итоговых значений
Пример расчета повыходных и агрегированных показателей по эфирным событиям и объединение результата в одну таблицу

## Описание задачи и условий расчета
- Регион: Россия 0+
- Период: 17.01.2022 - 23.01.2022
- ЦА: Все 4+, Все 14-44, M 25-34
- Место просмотра: Все места (дом+дача)
- Каналы: ТНТ
- Программы: категория - Игра-викторина, соревнование
- Статистики: Rtg000, RtgPer, Share

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике категорий программ найдем id категории Игра-викторина, соревнование
cats.get_tv_program_category(name=['Игра-викторина, соревнование'])

# Это id 6

In [ ]:
# Найдем id телекомпании ТНТ (СЕТЕВОЕ ВЕЩАНИЕ)
cats.get_tv_company(name=['ТНТ (СЕТЕВОЕ ВЕЩАНИЕ)'])

# id = 1870

## Расчет отдельных событий
Для расчета показателей по отдельным выходам посчитаем отчет События (Simple)

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2022-01-17', '2022-01-23')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы: ТНТ
company_filter = 'tvCompanyId = 1870'

# Указываем фильтр программ: категория - Игра-викторина, соревнование
program_filter = 'programCategoryId = 6'

# Фильтр блоков
break_filter = None

# Фильтр роликов
ad_filter = None

# Указываем список срезов (группировок)
slices = [
    'programTypeName',
    'programName',
    'programSpotId',
    'researchDate',
    'programStartTime',
    'programFinishTime'
]

# Указываем список статистик для расчета
statistics = ['Rtg000', 'RtgPer','Share']

# Задаем условия сортировки
sortings = None

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
}

Сформируем словарь с целевыми аудиториями

In [ ]:
# Задаем необходимые группы
targets = {
    'a. Все 4+':'age >= 4',
    'b. Все 14-44':'age >= 14 and age <= 44',
    'c. M 25-34':'sex = 1 and age >= 25 and age <= 34'
}

Посчитаем задания в цикле

In [ ]:
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задание и отправляем на расчет
for target, syntax in targets.items():
    
    # Подставляем значения словаря в параметры
    project_name = target 
    basedemo_filter = syntax
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_simple_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[['prj_name']+slices+statistics]

df

## Итоги
Агрегированные показатели посчитаем при помощи отчета Кросс-таблица (Crosstab)

### Итоги по программе
Фильтры идентичны повыходному расчету. Меняются срезы, статистики и опции расчета:

In [ ]:
# Указываем список срезов (группировок)
slices = [
    'programTypeName', #Программа жанр
    'programName' #Название программы
]

# Указываем список статистик для расчета
statistics = ['Rtg000Sum', 'RtgPerSum','ShareAvg']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "issueType": "PROGRAM", #Тип события - Программы
}

Посчитаем задания в цикле

In [ ]:
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задание и отправляем на расчет
for target, syntax in targets.items():
    
    # Подставляем значения словаря в параметры
    project_name = target 
    basedemo_filter = syntax
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_crosstab_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_crosstab_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df_prog = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df_prog = df_prog[['prj_name']+slices+statistics]

df_prog

In [ ]:
# Приведем названия статистик к названиям в simple отчете для корректного последующего объединения
df_prog.rename({'Rtg000Sum':'Rtg000', 'RtgPerSum':'RtgPer', 'ShareAvg':'Share'}, axis=1, inplace=True)
df_prog

### Итоги по жанру
В срезах оставляем только группировку по жанру:

In [ ]:
# Указываем список срезов (группировок)
slices = [
    'programTypeName', #Программа жанр
]

Посчитаем задания в цикле

In [ ]:
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задание и отправляем на расчет
for target, syntax in targets.items():
    
    # Подставляем значения словаря в параметры
    project_name = target 
    basedemo_filter = syntax
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_crosstab_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_crosstab_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df_genre = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df_genre = df_genre[['prj_name']+slices+statistics]

df_genre

In [ ]:
# Приведем названия статистик к названиям в simple отчете для корректного последующего объединения
df_genre.rename({'Rtg000Sum':'Rtg000', 'RtgPerSum':'RtgPer', 'ShareAvg':'Share'}, axis=1, inplace=True)
df_genre

## Объединение в одну таблицу

In [ ]:
# Соединяем три таблицы в одну
df_res = pd.concat([df, df_prog, df_genre], axis=0, join='outer')

# Задаем сортировку: ЦА, Программа, Дата, Время начала
df_res.sort_values(by=['prj_name','programName','researchDate','programStartTime'], inplace=True)

# Заменяем NaN на пустые ячейки
df_res = df_res.fillna('')

df_res

### Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
# Формируем сводную таблицу: строки - срезы, столбцы - ЦА, значения - статистики
df_res = pd.pivot_table(df_res, values=['Rtg000','RtgPer','Share'],
                        index=['programTypeName','programName','programSpotId','researchDate','programStartTime','programFinishTime'], 
                        columns=['prj_name'])
df_res

In [ ]:
# Опционально: поменять местами уровни в столбцах
df_res = df_res.swaplevel(0,1, axis=1).sort_index(axis=1)

df_res

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('04_events_with_totals')) as writer:
    df_res.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)